# Sentiment Analysis using RNN(Recurrent Neural Network)

In [0]:
import numpy as np
np.random.seed(7)

In [0]:
from keras.datasets import imdb

In [0]:
top_words=5000
(x_train,y_train),(x_test,y_test)=imdb.load_data(num_words=top_words)

In [26]:
x_train.shape

(25000,)

In [27]:
x_test.shape

(25000,)

In [28]:
y_train.shape

(25000,)

In [29]:
print(x_train[0])

[1, 14, 22, 16, 43, 530, 973, 1622, 1385, 65, 458, 4468, 66, 3941, 4, 173, 36, 256, 5, 25, 100, 43, 838, 112, 50, 670, 2, 9, 35, 480, 284, 5, 150, 4, 172, 112, 167, 2, 336, 385, 39, 4, 172, 4536, 1111, 17, 546, 38, 13, 447, 4, 192, 50, 16, 6, 147, 2025, 19, 14, 22, 4, 1920, 4613, 469, 4, 22, 71, 87, 12, 16, 43, 530, 38, 76, 15, 13, 1247, 4, 22, 17, 515, 17, 12, 16, 626, 18, 2, 5, 62, 386, 12, 8, 316, 8, 106, 5, 4, 2223, 2, 16, 480, 66, 3785, 33, 4, 130, 12, 16, 38, 619, 5, 25, 124, 51, 36, 135, 48, 25, 1415, 33, 6, 22, 12, 215, 28, 77, 52, 5, 14, 407, 16, 82, 2, 8, 4, 107, 117, 2, 15, 256, 4, 2, 7, 3766, 5, 723, 36, 71, 43, 530, 476, 26, 400, 317, 46, 7, 4, 2, 1029, 13, 104, 88, 4, 381, 15, 297, 98, 32, 2071, 56, 26, 141, 6, 194, 2, 18, 4, 226, 22, 21, 134, 476, 26, 480, 5, 144, 30, 2, 18, 51, 36, 28, 224, 92, 25, 104, 4, 226, 65, 16, 38, 1334, 88, 12, 16, 283, 5, 16, 4472, 113, 103, 32, 15, 16, 2, 19, 178, 32]


In [30]:
print(np.asarray(x_train[0]).shape)

(218,)


In [31]:
print(x_train[0][1])

14


In [0]:
dictw2c=imdb.get_word_index()
#print(dictw2c)

In [0]:
dict_c2w={i+3:w for (w,i) in dictw2c.items()}
#print(dict_c2w)

In [34]:
r1=x_train[0]
review={dict_c2w.get(c) for c in r1}
print(review)

{'i', 'what', 'imagine', 'definitely', 'big', 'remarks', 'play', None, 'now', 'were', 'released', 'casting', 'island', 'great', 'whole', "don't", 'throughout', 'say', 'played', 'fact', 'part', 'watch', 'end', "someone's", 'grown', 'should', 'would', 'after', 'lovely', 'cry', 'left', 'out', 'know', 'also', 'are', 'this', 'to', 'so', 'an', 'it', 'much', 'could', 'loved', 'came', 'that', 'list', 'myself', 'witty', 'at', 'be', 'you', 'connection', 'life', 'must', 'being', 'was', 'because', 'done', 'with', 'think', 'film', 'cried', 'location', 'these', 'a', 'really', 'scottish', 'norman', 'them', 'is', 'direction', 'fly', 'soon', 'and', 'actor', 'but', 'the', 'just', 'suited', 'robert', 'story', 'such', 'director', 'there', 'as', 'us', 'father', 'they', 'same', 'often', 'of', 'recommend', 'good', 'been', 'bought', 'stars', 'up', 'brilliant', 'children', 'real', 'from', 'sad', 'everyone', 'all', 'scenery', 'little', 'paul', "everyone's", 'if', 'two', 'for', 'amazing', 'true', 'have'}


In [0]:
from keras.preprocessing import sequence
max_review_len=500
X_train=sequence.pad_sequences(x_train,maxlen=max_review_len)
X_test=sequence.pad_sequences(x_test,maxlen=max_review_len)

In [14]:
X_train.shape

(25000, 500)

In [15]:
X_test.shape

(25000, 500)

In [16]:
print(X_train[0])

[   0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0 

In [0]:
from keras.models import Sequential
from keras.layers import Dense,LSTM,Conv1D,MaxPooling1D
from keras.layers.embeddings import Embedding

In [18]:
model=Sequential()
embedding_vector_len=32
model.add(Embedding(top_words,embedding_vector_len,input_length=max_review_len))

model.add(Conv1D(filters=32,kernel_size=3,activation="relu",padding='same'))
model.add(MaxPooling1D(pool_size=2))

model.add(LSTM(100, dropout=0.2, recurrent_dropout=0.2))

model.add(Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

model.summary()






Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 500, 32)           160000    
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 500, 32)           3104      
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, 250, 32)           0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 100)               53200     
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 101       
Total params: 216,405
Trainable para

In [19]:
model.fit(X_train,y_train,epochs=5,batch_size=64)
score=model.evaluate(X_test,y_test,verbose=0)
print(score)




Epoch 1/5





25000/25000 [==============================] - 227s 9ms/step - loss: 0.4600 - acc: 0.7698
Epoch 2/5
25000/25000 [==============================] - 213s 9ms/step - loss: 0.3141 - acc: 0.8721
Epoch 3/5
25000/25000 [==============================] - 214s 9ms/step - loss: 0.2624 - acc: 0.8948
Epoch 4/5
25000/25000 [==============================] - 213s 9ms/step - loss: 0.2301 - acc: 0.9113
Epoch 5/5
25000/25000 [==============================] - 214s 9ms/step - loss: 0.2064 - acc: 0.9207
[0.3094065869951248, 0.8734]


In [21]:
revPos="the movie is great and just brilliant casting is great"
revNeg="the movie is the worst and just boring story is slow"

rev1=revPos.split()
print(rev1)
rev2=[dictw2c.get(w)+3 for w in rev1]
rev3=[0]*(500-len(rev2)-1) +[1] +rev2
rev4=np.asarray(rev3).reshape(1,-1)
model.predict(rev4)

['the', 'movie', 'is', 'great', 'and', 'just', 'brilliant', 'casting', 'is', 'great']


array([[0.99406093]], dtype=float32)